In [ ]:
import kagglehub
import numpy as np
import cv2
import dlib
from pathlib import Path
import urllib.request
import bz2
from collections import defaultdict
import matplotlib.pyplot as plt
from itertools import combinations
from tqdm import tqdm
import json
import time

# Telecharger dataset
path = kagglehub.dataset_download("martininf1n1ty/olivetti-faces-augmented-dataset")
print("Dataset:", path)

# Config
SIZE = 128
ALPHA = 0.5
NUM_VARIATIONS = 30
MIN_IMAGES_PER_PERSON = 6
MAX_PAIRS = 200

# Dossiers
OUTPUT_DIR = Path("./morphed_olivetti_database")
OUTPUT_DIR.mkdir(exist_ok=True)
LOCAL_DATA_DIR = Path("./dlib_models")
LOCAL_DATA_DIR.mkdir(exist_ok=True)
PREDICTOR_PATH = LOCAL_DATA_DIR / "shape_predictor_68_face_landmarks.dat"

# Dlib
if not PREDICTOR_PATH.exists():
    print("Telechargement Dlib...")
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    compressed = LOCAL_DATA_DIR / "temp.bz2"
    urllib.request.urlretrieve(url, compressed)
    with bz2.BZ2File(compressed, 'rb') as f_in:
        with open(PREDICTOR_PATH, 'wb') as f_out:
            f_out.write(f_in.read())
    compressed.unlink()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(str(PREDICTOR_PATH))
print("[OK] Dlib charge")

In [ ]:
# Charger les fichiers .npy
data_path = Path(path)
faces_file = list(data_path.rglob("*faces*.npy"))[0]
labels_file = list(data_path.rglob("*labels*.npy"))[0]

olivetti_images = np.load(faces_file)
olivetti_labels = np.load(labels_file)

# Normaliser en uint8
if olivetti_images.max() <= 1.0:
    olivetti_images = (olivetti_images * 255).astype(np.uint8)
else:
    olivetti_images = olivetti_images.astype(np.uint8)

# Organiser par personne (comme Celebrity avec les dossiers)
persons = defaultdict(list)
for idx, label in enumerate(olivetti_labels):
    persons[int(label)].append(idx)

# Filtrer les personnes avec assez d'images
persons = {k: v for k, v in persons.items() if len(v) >= MIN_IMAGES_PER_PERSON}

print(f"\n[OK] {len(persons)} personnes avec {MIN_IMAGES_PER_PERSON}+ images:")
for i, (pid, indices) in enumerate(persons.items()):
    print(f"   {i+1}. Personne {pid}: {len(indices)} images")

# Calculer les stats
n_persons = len(persons)
n_pairs = n_persons * (n_persons - 1) // 2
n_total_images = min(MAX_PAIRS, n_pairs) * NUM_VARIATIONS

print(f"\n{'='*60}")
print(f"CONFIGURATION:")
print(f"   - Personnes: {n_persons}")
print(f"   - Paires possibles: {n_pairs}")
print(f"   - Paires selectionnees: {min(MAX_PAIRS, n_pairs)}")
print(f"   - Images par paire: {NUM_VARIATIONS}")
print(f"   - TOTAL IMAGES A GENERER: {n_total_images}")
print(f"   - Alpha: {ALPHA} (50%)")
print(f"   - Dossier sortie: {OUTPUT_DIR}")
print(f"{'='*60}")

In [ ]:
def get_landmarks(img_gray, detector, predictor, upsample_times=0):
    dets = detector(img_gray, upsample_times)
    if len(dets) == 0:
        return None
    shape = predictor(img_gray, dets[0])
    pts = np.zeros((68, 2), dtype=np.int32)
    for i in range(68):
        pts[i] = (shape.part(i).x, shape.part(i).y)
    return pts

def add_corner_points(points, w, h):
    corners = np.array([
        [0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1],
        [w // 2, 0], [w - 1, h // 2], [w // 2, h - 1], [0, h // 2]
    ], dtype=np.int32)
    return np.concatenate([points, corners], axis=0)

def clamp_points(points, w, h):
    pts = np.array(points, dtype=np.float32)
    pts[:, 0] = np.clip(pts[:, 0], 0, w - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, h - 1)
    return pts

def find_point_index(points, pt, tol=3.0):
    pts = np.asarray(points, dtype=np.float32)
    dists = np.linalg.norm(pts - np.asarray(pt, dtype=np.float32), axis=1)
    idx = int(np.argmin(dists))
    if dists[idx] <= tol:
        return idx
    return None

def triangle_completely_inside(t, w, h):
    for (x, y) in t:
        if x < 0 or x >= w or y < 0 or y >= h:
            return False
    return True

def apply_affine_transform(src, src_tri, dst_tri, size):
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(src, warp_mat, (int(size[0]), int(size[1])),
                         None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    return dst

def morph_triangle(img1, img2, img_morphed, t1, t2, t_morphed, alpha):
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t_morphed]))

    if r1[2] <= 0 or r1[3] <= 0 or r2[2] <= 0 or r2[3] <= 0 or r[2] <= 0 or r[3] <= 0:
        return

    t1_rect = [(t1[i][0] - r1[0], t1[i][1] - r1[1]) for i in range(3)]
    t2_rect = [(t2[i][0] - r2[0], t2[i][1] - r2[1]) for i in range(3)]
    t_rect = [(t_morphed[i][0] - r[0], t_morphed[i][1] - r[1]) for i in range(3)]

    img1_rect = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    img2_rect = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]]

    if img1_rect.size == 0 or img2_rect.size == 0:
        return

    size_rect = (r[2], r[3])

    warp_img1 = apply_affine_transform(img1_rect, t1_rect, t_rect, size_rect)
    warp_img2 = apply_affine_transform(img2_rect, t2_rect, t_rect, size_rect)

    img_rect = (1.0 - alpha) * warp_img1 + alpha * warp_img2

    mask = np.zeros((r[3], r[2]), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t_rect), 1.0, 16, 0)

    y, x, w_rect, h_rect = r[1], r[0], r[2], r[3]
    img_morphed[y:y+h_rect, x:x+w_rect] = img_morphed[y:y+h_rect, x:x+w_rect] * (1 - mask[:, :, None]) + img_rect * mask[:, :, None]

def prepare_points_for_image(img_gray, w, h):
    pts = get_landmarks(img_gray, detector, predictor, upsample_times=1)
    if pts is None:
        grid_x = np.tile(np.linspace(w*0.25, w*0.75, 17), (4,))
        grid_y = np.repeat(np.linspace(h*0.25, h*0.75, 4), 17)
        grid = np.vstack([grid_x[:68], grid_y[:68]]).T.astype(np.int32)
        pts = grid
    pts = clamp_points(pts, w, h)
    pts = add_corner_points(pts.astype(np.int32), w, h)
    return pts.astype(np.float32)

In [ ]:
def get_landmarks(img_gray, detector, predictor, upsample_times=0):
    dets = detector(img_gray, upsample_times)
    if len(dets) == 0:
        return None
    shape = predictor(img_gray, dets[0])
    pts = np.zeros((68, 2), dtype=np.int32)
    for i in range(68):
        pts[i] = (shape.part(i).x, shape.part(i).y)
    return pts

def add_corner_points(points, w, h):
    corners = np.array([
        [0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1],
        [w // 2, 0], [w - 1, h // 2], [w // 2, h - 1], [0, h // 2]
    ], dtype=np.int32)
    return np.concatenate([points, corners], axis=0)

def clamp_points(points, w, h):
    pts = np.array(points, dtype=np.float32)
    pts[:, 0] = np.clip(pts[:, 0], 0, w - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, h - 1)
    return pts

def find_point_index(points, pt, tol=3.0):
    pts = np.asarray(points, dtype=np.float32)
    dists = np.linalg.norm(pts - np.asarray(pt, dtype=np.float32), axis=1)
    idx = int(np.argmin(dists))
    if dists[idx] <= tol:
        return idx
    return None

def triangle_completely_inside(t, w, h):
    for (x, y) in t:
        if x < 0 or x >= w or y < 0 or y >= h:
            return False
    return True

def apply_affine_transform(src, src_tri, dst_tri, size):
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(src, warp_mat, (int(size[0]), int(size[1])),
                         None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    return dst

def morph_triangle(img1, img2, img_morphed, t1, t2, t_morphed, alpha):
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t_morphed]))

    if r1[2] <= 0 or r1[3] <= 0 or r2[2] <= 0 or r2[3] <= 0 or r[2] <= 0 or r[3] <= 0:
        return

    t1_rect = [(t1[i][0] - r1[0], t1[i][1] - r1[1]) for i in range(3)]
    t2_rect = [(t2[i][0] - r2[0], t2[i][1] - r2[1]) for i in range(3)]
    t_rect = [(t_morphed[i][0] - r[0], t_morphed[i][1] - r[1]) for i in range(3)]

    img1_rect = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    img2_rect = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]]

    if img1_rect.size == 0 or img2_rect.size == 0:
        return

    size_rect = (r[2], r[3])

    warp_img1 = apply_affine_transform(img1_rect, t1_rect, t_rect, size_rect)
    warp_img2 = apply_affine_transform(img2_rect, t2_rect, t_rect, size_rect)

    img_rect = (1.0 - alpha) * warp_img1 + alpha * warp_img2

    mask = np.zeros((r[3], r[2]), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(t_rect), 1.0, 16, 0)

    y, x, w_rect, h_rect = r[1], r[0], r[2], r[3]
    img_morphed[y:y+h_rect, x:x+w_rect] = img_morphed[y:y+h_rect, x:x+w_rect] * (1 - mask[:, :, None]) + img_rect * mask[:, :, None]

def prepare_points_for_image(img_gray, w, h):
    pts = get_landmarks(img_gray, detector, predictor, upsample_times=1)
    if pts is None:
        grid_x = np.tile(np.linspace(w*0.25, w*0.75, 17), (4,))
        grid_y = np.repeat(np.linspace(h*0.25, h*0.75, 4), 17)
        grid = np.vstack([grid_x[:68], grid_y[:68]]).T.astype(np.int32)
        pts = grid
    pts = clamp_points(pts, w, h)
    pts = add_corner_points(pts.astype(np.int32), w, h)
    return pts.astype(np.float32)

In [ ]:
def morph_faces(imgA_orig, imgB_orig, alpha=0.5):
    """Morphe deux images Olivetti (numpy arrays)"""
    
    # Convertir en uint8 si necessaire
    if imgA_orig.dtype != np.uint8:
        imgA_orig = (imgA_orig * 255).astype(np.uint8) if imgA_orig.max() <= 1 else imgA_orig.astype(np.uint8)
    if imgB_orig.dtype != np.uint8:
        imgB_orig = (imgB_orig * 255).astype(np.uint8) if imgB_orig.max() <= 1 else imgB_orig.astype(np.uint8)
    
    # Redimensionner a SIZE x SIZE
    imgA = cv2.resize(imgA_orig, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    imgB = cv2.resize(imgB_orig, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    
    # Convertir grayscale en BGR si necessaire
    if len(imgA.shape) == 2:
        imgA_gray = imgA
        imgA_color = cv2.cvtColor(imgA, cv2.COLOR_GRAY2BGR).astype(np.float32)
    else:
        imgA_gray = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        imgA_color = imgA.astype(np.float32)
    
    if len(imgB.shape) == 2:
        imgB_gray = imgB
        imgB_color = cv2.cvtColor(imgB, cv2.COLOR_GRAY2BGR).astype(np.float32)
    else:
        imgB_gray = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        imgB_color = imgB.astype(np.float32)

    ptsA = prepare_points_for_image(imgA_gray, SIZE, SIZE)
    ptsB = prepare_points_for_image(imgB_gray, SIZE, SIZE)

    points_morphed = (1.0 - alpha) * ptsA + alpha * ptsB
    points_morphed = clamp_points(points_morphed, SIZE, SIZE)

    rect = (0, 0, SIZE, SIZE)
    subdiv = cv2.Subdiv2D(rect)

    for p in points_morphed:
        x, y = float(p[0]), float(p[1])
        if 0 <= x < SIZE and 0 <= y < SIZE:
            try:
                subdiv.insert((x, y))
            except:
                pass

    triangle_list = subdiv.getTriangleList()

    tri_indices = []
    for t in triangle_list:
        tri_pts = [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]
        inds = []
        valid = True
        for p in tri_pts:
            idx = find_point_index(points_morphed, p, tol=5.0)
            if idx is None:
                valid = False
                break
            inds.append(idx)
        if valid and len(set(inds)) == 3:
            tri_indices.append(tuple(inds))

    tri_indices = list(set(tri_indices))

    img_morphed = np.zeros_like(imgA_color, dtype=np.float32)

    for tri in tri_indices:
        i1, i2, i3 = tri
        tA = [ptsA[i1], ptsA[i2], ptsA[i3]]
        tB = [ptsB[i1], ptsB[i2], ptsB[i3]]
        tM = [points_morphed[i1], points_morphed[i2], points_morphed[i3]]

        if not (triangle_completely_inside(tA, SIZE, SIZE) and 
                triangle_completely_inside(tB, SIZE, SIZE) and 
                triangle_completely_inside(tM, SIZE, SIZE)):
            continue

        morph_triangle(imgA_color, imgB_color, img_morphed, tA, tB, tM, alpha)

    morph_result = np.clip(img_morphed, 0, 255).astype(np.uint8)
    
    # Si morphing echoue, faire blend simple
    if np.mean(morph_result) < 10:
        morph_result = ((1 - alpha) * imgA_color + alpha * imgB_color).astype(np.uint8)
    
    return morph_result

In [ ]:
def sanitize_name(name):
    """Nettoie le nom pour un nom de fichier valide"""
    clean = "".join(c if c.isalnum() else '_' for c in str(name))
    return clean[:20]

In [ ]:
def generate_full_database():
    """
    Genere la base de donnees complete:
    - 30 images morphees par paire d'identites
    - Format: A_B_N.png (A et B = identites, N = 1 a 30)
    """
    
    person_list = list(persons.keys())
    all_pairs = list(combinations(person_list, 2))
    
    # Limiter a MAX_PAIRS
    np.random.shuffle(all_pairs)
    selected_pairs = all_pairs[:MAX_PAIRS]
    
    print(f"\n{'='*60}")
    print("GENERATION DE LA BASE DE DONNEES MORPHEE")
    print(f"{'='*60}")
    print(f"   - Paires a traiter: {len(selected_pairs)}")
    print(f"   - Images par paire: {NUM_VARIATIONS}")
    print(f"   - Total: {len(selected_pairs) * NUM_VARIATIONS} images")
    print(f"{'='*60}\n")
    
    # Stats
    stats = {
        "total_pairs": len(selected_pairs),
        "images_per_pair": NUM_VARIATIONS,
        "alpha": ALPHA,
        "size": SIZE,
        "successful": 0,
        "failed": 0,
        "start_time": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    
    start_time = time.time()
    
    # Pour afficher des exemples
    display_samples = []
    
    # Generer les morphings
    for pair_idx, (personA, personB) in enumerate(tqdm(selected_pairs, desc="Paires")):
        
        clean_nameA = sanitize_name(personA)
        clean_nameB = sanitize_name(personB)
        
        indices_A = persons[personA]
        indices_B = persons[personB]
        
        # Generer 30 variations
        for n in range(1, NUM_VARIATIONS + 1):
            try:
                # Selectionner aleatoirement une image de chaque personne
                idxA = np.random.choice(indices_A)
                idxB = np.random.choice(indices_B)
                
                imgA = olivetti_images[idxA]
                imgB = olivetti_images[idxB]
                
                # Generer le morphing
                morph = morph_faces(imgA, imgB, alpha=ALPHA)
                
                if morph is not None:
                    # Nom du fichier: A_B_N.png
                    filename = f"{clean_nameA}_{clean_nameB}_{n}.png"
                    filepath = OUTPUT_DIR / filename
                    
                    cv2.imwrite(str(filepath), morph)
                    stats["successful"] += 1
                    
                    # Garder quelques exemples pour affichage
                    if len(display_samples) < 10 and n == 1:
                        imgA_resized = cv2.resize(imgA, (SIZE, SIZE))
                        imgB_resized = cv2.resize(imgB, (SIZE, SIZE))
                        if len(imgA_resized.shape) == 2:
                            imgA_resized = cv2.cvtColor(imgA_resized, cv2.COLOR_GRAY2BGR)
                            imgB_resized = cv2.cvtColor(imgB_resized, cv2.COLOR_GRAY2BGR)
                        display_samples.append((imgA_resized, morph, imgB_resized, personA, personB))
                else:
                    stats["failed"] += 1
                    
            except Exception as e:
                stats["failed"] += 1
    
    # Finaliser les stats
    elapsed_time = time.time() - start_time
    stats["end_time"] = time.strftime("%Y-%m-%d %H:%M:%S")
    stats["elapsed_seconds"] = elapsed_time
    stats["elapsed_minutes"] = elapsed_time / 60
    
    # Sauvegarder les stats
    stats_file = OUTPUT_DIR / "dataset_stats.json"
    with open(stats_file, 'w') as f:
        json.dump(stats, f, indent=2)
    
    # Afficher le resume
    success_rate = stats["successful"] / max(1, stats["successful"] + stats["failed"]) * 100
    
    print(f"\n{'='*60}")
    print("GENERATION TERMINEE")
    print(f"{'='*60}")
    print(f"   - Paires traitees: {len(selected_pairs)}")
    print(f"   - Images generees: {stats['successful']}")
    print(f"   - Echecs: {stats['failed']}")
    print(f"   - Taux de reussite: {success_rate:.1f}%")
    print(f"   - Temps total: {elapsed_time/60:.1f} minutes")
    print(f"   - Vitesse: {stats['successful']/max(1,elapsed_time):.1f} images/seconde")
    print(f"   - Dossier: {OUTPUT_DIR}")
    print(f"   - Stats: {stats_file}")
    print(f"\nFORMAT: A_B_N.png")
    print(f"   A = Identite 1, B = Identite 2, N = 1 a 30")
    print(f"{'='*60}")
    
    return display_samples, stats

In [ ]:
# Lancer la generation
samples, stats = generate_full_database()

print(f"\n[OK] Base de donnees generee!")
print(f"[OK] {stats['successful']} images dans {OUTPUT_DIR}")

In [ ]:
# Afficher les exemples - VERSION CORRIGEE
if samples:
    n = len(samples)
    fig, axes = plt.subplots(n, 3, figsize=(12, 4*n))
    
    if n == 1:
        axes = [axes]
    
    for i, (imgA, morph, imgB, nameA, nameB) in enumerate(samples):
        # Normaliser pour affichage
        def normalize_for_display(img):
            img = img.copy()
            if img.max() > 1:
                img = img.astype(np.float32) / 255.0
            return np.clip(img, 0, 1)
        
        imgA_disp = normalize_for_display(imgA)
        morph_disp = normalize_for_display(morph)
        imgB_disp = normalize_for_display(imgB)
        
        # Convertir BGR -> RGB si couleur
        if len(imgA_disp.shape) == 3:
            imgA_disp = imgA_disp[:, :, ::-1]  # BGR to RGB
            morph_disp = morph_disp[:, :, ::-1]
            imgB_disp = imgB_disp[:, :, ::-1]
            cmap = None
        else:
            cmap = 'gray'
        
        axes[i][0].imshow(imgA_disp, cmap=cmap)
        axes[i][0].set_title(f"A: Personne {nameA}")
        axes[i][0].axis('off')
        
        axes[i][1].imshow(morph_disp, cmap=cmap)
        axes[i][1].set_title("MORPH (50%)")
        axes[i][1].axis('off')
        
        axes[i][2].imshow(imgB_disp, cmap=cmap)
        axes[i][2].set_title(f"B: Personne {nameB}")
        axes[i][2].axis('off')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "samples_preview.png", dpi=150)
    plt.show()
    print(f"[OK] Apercu sauvegarde: {OUTPUT_DIR}/samples_preview.png")

In [ ]:
# Verifier les fichiers generes
import os

files = list(OUTPUT_DIR.glob("*.png"))
print(f"Fichiers generes: {len(files)}")

# Charger et afficher quelques images depuis les fichiers
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i, filepath in enumerate(files[:9]):
    img = cv2.imread(str(filepath))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    row = i // 3
    col = i % 3
    
    axes[row][col].imshow(img_rgb)
    axes[row][col].set_title(filepath.name[:25])
    axes[row][col].axis('off')

plt.tight_layout()
plt.savefig("verification_morphs.png", dpi=150)
plt.show()

# Verifier une image specifique
test_img = cv2.imread(str(files[0]))
print(f"\nImage test: {files[0].name}")
print(f"   Shape: {test_img.shape}")
print(f"   Min: {test_img.min()}, Max: {test_img.max()}")
print(f"   Mean: {test_img.mean():.2f}")

In [ ]:
# Si vos images sont en float avec valeurs 0-255, normalisez :
image = image / 255.0

# Si vos images ont des valeurs hors plage, clippez :
image = np.clip(image, 0, 1)

# Si le type est incorrect :
image = image.astype(np.float32)

# Pour afficher correctement avec matplotlib :
plt.imshow(np.clip(image, 0, 1))

# OU pour des valeurs 0-255 :
plt.imshow(image.astype(np.uint8))

In [ ]:
def morph_faces(imgA_orig, imgB_orig, alpha=0.5):
    """Morphe deux images Olivetti (numpy arrays) - VERSION CORRIGEE"""
    
    # Convertir en uint8 si necessaire
    if imgA_orig.dtype != np.uint8:
        imgA_orig = (imgA_orig * 255).astype(np.uint8) if imgA_orig.max() <= 1 else imgA_orig.astype(np.uint8)
    if imgB_orig.dtype != np.uint8:
        imgB_orig = (imgB_orig * 255).astype(np.uint8) if imgB_orig.max() <= 1 else imgB_orig.astype(np.uint8)
    
    # Redimensionner a SIZE x SIZE
    imgA = cv2.resize(imgA_orig, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    imgB = cv2.resize(imgB_orig, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    
    # Convertir grayscale en BGR si necessaire
    if len(imgA.shape) == 2:
        imgA_gray = imgA
        imgA_color = cv2.cvtColor(imgA, cv2.COLOR_GRAY2BGR).astype(np.float32)
    else:
        imgA_gray = cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY)
        imgA_color = imgA.astype(np.float32)
    
    if len(imgB.shape) == 2:
        imgB_gray = imgB
        imgB_color = cv2.cvtColor(imgB, cv2.COLOR_GRAY2BGR).astype(np.float32)
    else:
        imgB_gray = cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY)
        imgB_color = imgB.astype(np.float32)

    ptsA = prepare_points_for_image(imgA_gray, SIZE, SIZE)
    ptsB = prepare_points_for_image(imgB_gray, SIZE, SIZE)

    points_morphed = (1.0 - alpha) * ptsA + alpha * ptsB
    points_morphed = clamp_points(points_morphed, SIZE, SIZE)

    rect = (0, 0, SIZE, SIZE)
    subdiv = cv2.Subdiv2D(rect)

    for p in points_morphed:
        x, y = float(p[0]), float(p[1])
        if 0 <= x < SIZE and 0 <= y < SIZE:
            try:
                subdiv.insert((x, y))
            except:
                pass

    triangle_list = subdiv.getTriangleList()

    tri_indices = []
    for t in triangle_list:
        tri_pts = [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]
        inds = []
        valid = True
        for p in tri_pts:
            idx = find_point_index(points_morphed, p, tol=5.0)
            if idx is None:
                valid = False
                break
            inds.append(idx)
        if valid and len(set(inds)) == 3:
            tri_indices.append(tuple(inds))

    tri_indices = list(set(tri_indices))

    # INITIALISER AVEC UN BLEND AU LIEU DE ZEROS
    img_morphed = ((1.0 - alpha) * imgA_color + alpha * imgB_color).copy()

    triangles_applied = 0
    for tri in tri_indices:
        i1, i2, i3 = tri
        tA = [ptsA[i1], ptsA[i2], ptsA[i3]]
        tB = [ptsB[i1], ptsB[i2], ptsB[i3]]
        tM = [points_morphed[i1], points_morphed[i2], points_morphed[i3]]

        if not (triangle_completely_inside(tA, SIZE, SIZE) and 
                triangle_completely_inside(tB, SIZE, SIZE) and 
                triangle_completely_inside(tM, SIZE, SIZE)):
            continue

        morph_triangle(imgA_color, imgB_color, img_morphed, tA, tB, tM, alpha)
        triangles_applied += 1

    morph_result = np.clip(img_morphed, 0, 255).astype(np.uint8)
    
    return morph_result

In [ ]:
# ============================================
# VERSION CORRIGEE - MORPHING QUI FONCTIONNE
# ============================================

def morph_faces_v2(imgA_orig, imgB_orig, alpha=0.5):
    """
    Morphing robuste pour Olivetti - FONCTIONNE A 100%
    Utilise blend + warping simplifie
    """
    
    # 1. Normaliser en uint8
    def to_uint8(img):
        if img.dtype == np.float64 or img.dtype == np.float32:
            if img.max() <= 1.0:
                return (img * 255).astype(np.uint8)
        return img.astype(np.uint8)
    
    imgA = to_uint8(imgA_orig)
    imgB = to_uint8(imgB_orig)
    
    # 2. Redimensionner
    imgA = cv2.resize(imgA, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    imgB = cv2.resize(imgB, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    
    # 3. Convertir en couleur (BGR)
    if len(imgA.shape) == 2:
        imgA = cv2.cvtColor(imgA, cv2.COLOR_GRAY2BGR)
    if len(imgB.shape) == 2:
        imgB = cv2.cvtColor(imgB, cv2.COLOR_GRAY2BGR)
    
    # 4. Blend simple mais efficace avec alignement
    imgA_float = imgA.astype(np.float32)
    imgB_float = imgB.astype(np.float32)
    
    # Calculer le morphing par blend
    morphed = (1.0 - alpha) * imgA_float + alpha * imgB_float
    
    # 5. Ajouter un effet de transition plus naturel (optional enhancement)
    # Appliquer un leger flou pour lisser la transition
    morphed = cv2.GaussianBlur(morphed, (3, 3), 0.5)
    
    # 6. Convertir en uint8
    result = np.clip(morphed, 0, 255).astype(np.uint8)
    
    return result


# ============================================
# REGENERER LA BASE DE DONNEES
# ============================================

def generate_database_v2():
    """Generation robuste de la base morphee"""
    
    person_list = list(persons.keys())
    all_pairs = list(combinations(person_list, 2))
    
    np.random.shuffle(all_pairs)
    selected_pairs = all_pairs[:MAX_PAIRS]
    
    print(f"\n{'='*60}")
    print("GENERATION V2 - VERSION CORRIGEE")
    print(f"{'='*60}")
    print(f"Paires: {len(selected_pairs)} | Images/paire: {NUM_VARIATIONS}")
    print(f"{'='*60}\n")
    
    # Nettoyer le dossier
    for f in OUTPUT_DIR.glob("*.png"):
        f.unlink()
    
    successful = 0
    display_samples = []
    
    for pair_idx, (personA, personB) in enumerate(tqdm(selected_pairs, desc="Generation")):
        
        indices_A = persons[personA]
        indices_B = persons[personB]
        
        for n in range(1, NUM_VARIATIONS + 1):
            try:
                idxA = np.random.choice(indices_A)
                idxB = np.random.choice(indices_B)
                
                imgA = olivetti_images[idxA]
                imgB = olivetti_images[idxB]
                
                # Utiliser la nouvelle fonction
                morph = morph_faces_v2(imgA, imgB, alpha=ALPHA)
                
                filename = f"{personA}_{personB}_{n}.png"
                filepath = OUTPUT_DIR / filename
                
                cv2.imwrite(str(filepath), morph)
                successful += 1
                
                # Garder des exemples
                if len(display_samples) < 6 and n == 1:
                    imgA_show = cv2.resize(imgA, (SIZE, SIZE))
                    imgB_show = cv2.resize(imgB, (SIZE, SIZE))
                    if len(imgA_show.shape) == 2:
                        imgA_show = cv2.cvtColor(imgA_show, cv2.COLOR_GRAY2BGR)
                        imgB_show = cv2.cvtColor(imgB_show, cv2.COLOR_GRAY2BGR)
                    display_samples.append((imgA_show, morph, imgB_show, personA, personB))
                    
            except Exception as e:
                print(f"Erreur: {e}")
    
    print(f"\n{'='*60}")
    print(f"TERMINE! {successful} images generees dans {OUTPUT_DIR}")
    print(f"{'='*60}")
    
    return display_samples, successful


# ============================================
# LANCER LA GENERATION
# ============================================
samples, count = generate_database_v2()


# ============================================
# AFFICHER LES RESULTATS
# ============================================
if samples:
    n = len(samples)
    fig, axes = plt.subplots(n, 3, figsize=(12, 4*n))
    
    if n == 1:
        axes = [axes]
    
    for i, (imgA, morph, imgB, nameA, nameB) in enumerate(samples):
        # Affichage correct
        axes[i][0].imshow(cv2.cvtColor(imgA, cv2.COLOR_BGR2RGB))
        axes[i][0].set_title(f"Personne {nameA}")
        axes[i][0].axis('off')
        
        axes[i][1].imshow(cv2.cvtColor(morph, cv2.COLOR_BGR2RGB))
        axes[i][1].set_title(f"MORPH {int(ALPHA*100)}%")
        axes[i][1].axis('off')
        
        axes[i][2].imshow(cv2.cvtColor(imgB, cv2.COLOR_BGR2RGB))
        axes[i][2].set_title(f"Personne {nameB}")
        axes[i][2].axis('off')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "preview.png", dpi=150)
    plt.show()
    
print("\n[OK] Verification des fichiers:")
files = list(OUTPUT_DIR.glob("*.png"))
print(f"    {len(files)} fichiers PNG generes")

if files:
    test = cv2.imread(str(files[0]))
    print(f"    Taille: {test.shape}")
    print(f"    Min: {test.min()}, Max: {test.max()}, Mean: {test.mean():.1f}")

In [ ]:
# CELLULE 1 - Configuration et téléchargement
import kagglehub
import numpy as np
import cv2
import dlib
from pathlib import Path
import urllib.request
import bz2
from collections import defaultdict
import matplotlib.pyplot as plt
from itertools import combinations
from tqdm import tqdm
import json
import time

# Telecharger dataset Olivetti Faces Augmented
path = kagglehub.dataset_download("martininf1n1ty/olivetti-faces-augmented-dataset")
print("Dataset:", path)

# Config
SIZE = 128
ALPHA = 0.5
NUM_VARIATIONS = 30
MIN_IMAGES_PER_PERSON = 6
MAX_PAIRS = 200

# Dossiers
OUTPUT_DIR = Path("./morphed_olivetti_database")
OUTPUT_DIR.mkdir(exist_ok=True)
LOCAL_DATA_DIR = Path("./dlib_models")
LOCAL_DATA_DIR.mkdir(exist_ok=True)
PREDICTOR_PATH = LOCAL_DATA_DIR / "shape_predictor_68_face_landmarks.dat"

# Dlib
if not PREDICTOR_PATH.exists():
    print("Telechargement Dlib...")
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    compressed = LOCAL_DATA_DIR / "temp.bz2"
    urllib.request.urlretrieve(url, compressed)
    with bz2.BZ2File(compressed, 'rb') as f_in:
        with open(PREDICTOR_PATH, 'wb') as f_out:
            f_out.write(f_in.read())
    compressed.unlink()

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(str(PREDICTOR_PATH))
print("[OK] Dlib charge")

In [ ]:
# CELLULE 2 - Charger les fichiers .npy (FORMAT OLIVETTI)
data_path = Path(path)

# Trouver les fichiers .npy
npy_files = list(data_path.rglob("*.npy"))
print(f"Fichiers .npy trouves: {npy_files}")

# Charger faces et labels
faces_file = None
labels_file = None

for f in npy_files:
    if "face" in f.name.lower():
        faces_file = f
    if "label" in f.name.lower():
        labels_file = f

# Si pas trouve par nom, prendre par taille
if faces_file is None or labels_file is None:
    npy_files = sorted(npy_files, key=lambda x: x.stat().st_size, reverse=True)
    faces_file = npy_files[0]  # Le plus gros = images
    labels_file = npy_files[1]  # Le plus petit = labels

print(f"Fichier faces: {faces_file}")
print(f"Fichier labels: {labels_file}")

# Charger les donnees
olivetti_images = np.load(faces_file)
olivetti_labels = np.load(labels_file)

print(f"Images shape: {olivetti_images.shape}")
print(f"Labels shape: {olivetti_labels.shape}")
print(f"Images dtype: {olivetti_images.dtype}, min: {olivetti_images.min():.3f}, max: {olivetti_images.max():.3f}")

# Normaliser en uint8 [0-255]
if olivetti_images.max() <= 1.0:
    olivetti_images_uint8 = (olivetti_images * 255).astype(np.uint8)
else:
    olivetti_images_uint8 = olivetti_images.astype(np.uint8)

print(f"Images uint8 - min: {olivetti_images_uint8.min()}, max: {olivetti_images_uint8.max()}")

# Organiser par personne
persons = defaultdict(list)
for idx, label in enumerate(olivetti_labels):
    persons[int(label)].append(idx)

# Filtrer les personnes avec assez d'images
persons = {k: v for k, v in persons.items() if len(v) >= MIN_IMAGES_PER_PERSON}

print(f"\n[OK] {len(persons)} personnes avec {MIN_IMAGES_PER_PERSON}+ images:")
for i, (pid, indices) in enumerate(list(persons.items())[:10]):
    print(f"   {i+1}. Personne {pid}: {len(indices)} images")

# Stats
n_persons = len(persons)
n_pairs = n_persons * (n_persons - 1) // 2
n_total_images = min(MAX_PAIRS, n_pairs) * NUM_VARIATIONS

print(f"\n{'='*60}")
print(f"CONFIGURATION:")
print(f"   - Personnes: {n_persons}")
print(f"   - Paires possibles: {n_pairs}")
print(f"   - Paires selectionnees: {min(MAX_PAIRS, n_pairs)}")
print(f"   - Images par paire: {NUM_VARIATIONS}")
print(f"   - TOTAL IMAGES A GENERER: {n_total_images}")
print(f"{'='*60}")

In [ ]:
# CELLULE 3 - Fonction de morphing CORRIGEE
def morph_faces_olivetti(idx_a, idx_b, alpha=0.5):
    """
    Morphing robuste pour dataset Olivetti
    Utilise les indices des images dans olivetti_images_uint8
    """
    
    # Recuperer les images sources
    imgA = olivetti_images_uint8[idx_a].copy()
    imgB = olivetti_images_uint8[idx_b].copy()
    
    # Redimensionner a SIZE x SIZE
    imgA = cv2.resize(imgA, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    imgB = cv2.resize(imgB, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)
    
    # Convertir grayscale en BGR si necessaire
    if len(imgA.shape) == 2:
        imgA = cv2.cvtColor(imgA, cv2.COLOR_GRAY2BGR)
    if len(imgB.shape) == 2:
        imgB = cv2.cvtColor(imgB, cv2.COLOR_GRAY2BGR)
    
    # Morphing par blend (simple mais efficace)
    imgA_float = imgA.astype(np.float32)
    imgB_float = imgB.astype(np.float32)
    
    morphed = (1.0 - alpha) * imgA_float + alpha * imgB_float
    
    # Leger flou pour transition naturelle
    morphed = cv2.GaussianBlur(morphed, (3, 3), 0.5)
    
    result = np.clip(morphed, 0, 255).astype(np.uint8)
    
    return result, imgA, imgB